In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
energy_grid = pd.read_csv('Data_for_UCI_named.csv')

In [3]:
energy_grid.head()

tau1      tau2      tau3      tau4        p1        p2        p3  \
0  2.959060  3.079885  8.381025  9.780754  3.763085 -0.782604 -1.257395   
1  9.304097  4.902524  3.047541  1.369357  5.067812 -1.940058 -1.872742   
2  8.971707  8.848428  3.046479  1.214518  3.405158 -1.207456 -1.277210   
3  0.716415  7.669600  4.486641  2.340563  3.963791 -1.027473 -1.938944   
4  3.134112  7.608772  4.943759  9.857573  3.525811 -1.125531 -1.845975   

         p4        g1        g2        g3        g4      stab     stabf  
0 -1.723086  0.650456  0.859578  0.887445  0.958034  0.055347  unstable  
1 -1.255012  0.413441  0.862414  0.562139  0.781760 -0.005957    stable  
2 -0.920492  0.163041  0.766689  0.839444  0.109853  0.003471  unstable  
3 -0.997374  0.446209  0.976744  0.929381  0.362718  0.028871  unstable  
4 -0.554305  0.797110  0.455450  0.656947  0.820923  0.049860  unstable

In [4]:
X = energy_grid.drop(columns = ['stabf'])
y = energy_grid['stabf']

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_scaler = scaler.fit_transform(x_train)
x_test_scaler = scaler.transform(x_test)

In [13]:
!pip install xgboost

In [7]:
#Importing XGBoost Classifier

from xgboost import XGBClassifier
XGB = XGBClassifier(random_state = 1)

XGB.fit(x_train_scaler, y_train)

XGB_pred = XGB.predict(x_test_scaler)

from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

XGB_accuracy = accuracy_score(y_test,XGB_pred)
print('Accuracy: {}'.format(XGB_accuracy))

C:\Users\HP\anaconda3\envs\Hamoye\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:51:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.9995


In [8]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state = 1)

RFC.fit(x_train_scaler, y_train)

RFC_pred = RFC.predict(x_test_scaler)

RFC_accuracy = accuracy_score(y_test, RFC_pred)
print('Accuracy: {}'.format(RFC_accuracy))

Accuracy: 0.9995


In [11]:
!pip install lightgbm

In [13]:
import lightgbm
from lightgbm import LGBMClassifier

In [14]:
LGBM = LGBMClassifier(random_state = 1)

LGBM.fit(x_train_scaler, y_train)

LGBM_pred = LGBM.predict(x_test_scaler)

LGBM_accuracy = accuracy_score(y_test, LGBM_pred)
print('Accuracy: {}'.format(LGBM_accuracy))

Accuracy: 0.9965


In [15]:
from sklearn.ensemble import ExtraTreesClassifier
ETC = ExtraTreesClassifier(random_state = 1)

ETC.fit(x_train_scaler, y_train)

ETC_pred = ETC.predict(x_test_scaler)

ETC_accuracy = accuracy_score(y_test, ETC_pred)
print('Accuracy: {}'.format(ETC_accuracy))



Accuracy: 0.993


In [21]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split, 'max_features': max_features}

from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(ETC, hyperparameter_grid, cv=5, n_iter=10, scoring='accuracy', n_jobs = -1, verbose = 1, random_state = 1)

searchCV = clf.fit(x_train_scaler, y_train)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [22]:
searchCV.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [23]:
ETC1 = ExtraTreesClassifier(max_features=None,min_samples_leaf=8,min_samples_split=2,n_estimators=1000,random_state=1)
ETC1.fit(x_train_scaler, y_train)
ETC1_pred = ETC1.predict(x_test_scaler)

ETC1_accuracy = accuracy_score(y_true=y_test, y_pred=ETC1_pred)
print('Accuracy: {}'.format(ETC1_accuracy))

Accuracy: 0.9995


In [24]:
feature = X.columns

feat_importance = pd.DataFrame(ETC.feature_importances_,index=feature)
feat = feat_importance.sort_values(0)
feat

0
p2    0.016284
p1    0.016530
p4    0.016692
p3    0.016735
g1    0.039116
g4    0.042511
g2    0.043428
g3    0.044304
tau1  0.053642
tau3  0.057159
tau2  0.057985
tau4  0.058033
stab  0.537580

In [25]:
print('most important feature: {}'.format(feat.idxmax()))

print('least important feature: {}'.format(feat.idxmin()))

most important feature: 0    stab
dtype: object
least important feature: 0    p2
dtype: object
